In [1]:
import os
import numpy as np
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Get required data
From the github repository

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/XLipcak/rev-rec.git cloned-repo

fatal: destination path 'cloned-repo' already exists and is not an empty directory.


In [ ]:
%ls

cloned-repo/  drive/  openstack.json  sample_data/


In [ ]:
# Save OpenStack dataset
import shutil
src = "/content/cloned-repo/rev-rec-data/openstack.json"
dst = "/content/drive/MyDrive/Colab Notebooks/My Project/openstack.json"
shutil.move(src, dst)

FileNotFoundError: ignored

# Read json file

In [ ]:
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/openstack.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'subProject': 'openstack/tempest', 'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d', 'changeNumber': 6139, 'timestamp': 1333428006000, 'reviewers': [{'accountId': '97', 'email': 'daryl.walleck@rackspace.com', 'name': 'Daryl Walleck', 'avatar': None}, {'accountId': '7', 'email': 'jaypipes@gmail.com', 'name': 'Jay Pipes', 'avatar': None}], 'owner': {'accountId': '2238', 'email': 'rajalakshmi.girish@hpe.com', 'name': 'rajalakshmi-ganesan', 'avatar': None}, 'filePaths': [{'location': 'tempest/tests/test_volumes_get.py'}]}
{'subProject': 'openstack/tempest', 'changeId': 'I10e7116570f922ec87e23b1f880cd4b1c08c3088', 'changeNumber': 7438, 'timestamp': 1337072165000, 'reviewers': [{'accountId': '97', 'email': 'daryl.walleck@rackspace.com', 'name': 'Daryl Walleck', 'avatar': None}, {'accountId': '7', 'email': 'jaypipes@gmail.com', 'name': 'Jay Pipes', 'avatar': None}], 'owner': {'accountId': '4120', 'email': 'rohit.karajgi@ril.com', 'name': 'Rohit Karajgi', 'avatar': None}, 'filePaths': 

## Hands on with given data

In [ ]:
type(data)

list

In [ ]:
len(data)

6545

In [ ]:
# get an element of the list of data
data[0]

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
 'changeNumber': 6139,
 'filePaths': [{'location': 'tempest/tests/test_volumes_get.py'}],
 'owner': {'accountId': '2238',
  'avatar': None,
  'email': 'rajalakshmi.girish@hpe.com',
  'name': 'rajalakshmi-ganesan'},
 'reviewers': [{'accountId': '97',
   'avatar': None,
   'email': 'daryl.walleck@rackspace.com',
   'name': 'Daryl Walleck'},
  {'accountId': '7',
   'avatar': None,
   'email': 'jaypipes@gmail.com',
   'name': 'Jay Pipes'}],
 'subProject': 'openstack/tempest',
 'timestamp': 1333428006000}

In [ ]:
# read a single feature of an element
data[0]['reviewers']

[{'accountId': '97',
  'avatar': None,
  'email': 'daryl.walleck@rackspace.com',
  'name': 'Daryl Walleck'},
 {'accountId': '7',
  'avatar': None,
  'email': 'jaypipes@gmail.com',
  'name': 'Jay Pipes'}]

In [ ]:
# Get the Time of available data from selected project
from datetime import datetime
  
timestamp = data[0]['timestamp']
dt_obj = datetime.fromtimestamp(1333428006).strftime('%y-%m-%d')  # should drop the 3 last zero digits
print("End   time:", timestamp, "is equal to date:", dt_obj)

timestamp = data[6544]['timestamp']
dt_obj = datetime.fromtimestamp(1311695026).strftime('%y-%m-%d')
print("Start time:", timestamp, "is equal to date:", dt_obj)

End   time: 1333428006000 is equal to date: 12-04-03
Start time: 1311695026000 is equal to date: 11-07-26


The openstack data that we use in here, containes about 8 months information of openstack project (2011-2012)

## Cleaning data

In [ ]:
# Extract and save desired features from each pull request. The new desired data is new_data
new_data = []

for i in range(0, len(data)):
  elem_dict = {"changeId":data[i]['changeId'], "reviewers":data[i]['reviewers'], "owner":data[i]['owner'], "filePaths":data[i]['filePaths']}
  new_data.append(elem_dict)
  # print(new_data[i])

print(len(new_data))

6545


In [ ]:
new_data[0]

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
 'filePaths': [{'location': 'tempest/tests/test_volumes_get.py'}],
 'owner': {'accountId': '2238',
  'avatar': None,
  'email': 'rajalakshmi.girish@hpe.com',
  'name': 'rajalakshmi-ganesan'},
 'reviewers': [{'accountId': '97',
   'avatar': None,
   'email': 'daryl.walleck@rackspace.com',
   'name': 'Daryl Walleck'},
  {'accountId': '7',
   'avatar': None,
   'email': 'jaypipes@gmail.com',
   'name': 'Jay Pipes'}]}

In [ ]:
# Remove remaining undesired features. The final desired information is saved in final_data
final_data = []

for i in range(0, len(new_data)):
  elem_dict = {"changeId":new_data[i]['changeId'], "ownerId": new_data[i]['owner']['accountId'], "reviewersId":[], "filePaths":[]}

  for j in range(0, len(new_data[i]['reviewers'])):
    elem_dict["reviewersId"].append(new_data[i]['reviewers'][j]['accountId'])

  for j in range(0, len(new_data[i]['filePaths'])):
    elem_dict["filePaths"].append(new_data[i]['filePaths'][j]['location'])
  
  final_data.append(elem_dict)
  # print(final_data[i])

print(len(final_data))

6545


In [ ]:
final_data[1]

{'changeId': 'I10e7116570f922ec87e23b1f880cd4b1c08c3088',
 'filePaths': ['tempest/tests/identity/test_roles.py',
  'tempest/services/identity/json/admin_client.py',
  'tempest/tests/identity/base_admin_test.py'],
 'ownerId': '4120',
 'reviewersId': ['97', '7']}

In [ ]:
type(final_data)

list

In [ ]:
# Convert list to json
# It should be a string before write it to file
json_string = json.dumps(final_data)

# Save OpenStack new dataset
file = open('openstack_final.json', 'w')
file.write(json_string)
file.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# Move OpenStack dataset to proper location
import shutil

src = "./openstack_final.json"
dst = "/content/drive/MyDrive/Colab Notebooks/My Project/openstack_final.json"
shutil.move(src, dst)

'/content/drive/MyDrive/Colab Notebooks/My Project/openstack_final.json'

# Approach

## Load final data

In [3]:
# Load final data from drive
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/openstack_final.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d', 'ownerId': '2238', 'reviewersId': ['97', '7'], 'filePaths': ['tempest/tests/test_volumes_get.py']}
{'changeId': 'I10e7116570f922ec87e23b1f880cd4b1c08c3088', 'ownerId': '4120', 'reviewersId': ['97', '7'], 'filePaths': ['tempest/tests/identity/test_roles.py', 'tempest/services/identity/json/admin_client.py', 'tempest/tests/identity/base_admin_test.py']}
{'changeId': 'I1d0c395cf679e07e304dd878d97734307b685f4c', 'ownerId': '1849', 'reviewersId': ['679', '1561'], 'filePaths': ['.gitignore']}
{'changeId': 'Ia151f265bf62e6704de6cc2f2c01963be9e2dd69', 'ownerId': '1849', 'reviewersId': ['1561', '357'], 'filePaths': ['tools/hacking.py']}
{'changeId': 'Ie4cfec6f89d6b37554b5345cde92f982397915e2', 'ownerId': '2', 'reviewersId': ['1561', '357'], 'filePaths': ['tox.ini']}


## Split data for train/test sets

## Main components of the approach
Extract 3 interaction metrices